In [1]:
import random
import argparse
import os
import openai
import time
import re
import json
import multiprocessing
import numpy as np
import torch
from statistics import mean

import datasets
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset

from utils import create_demo_text, MyDataset, setup_data_loader, parse_arguments
from decoder import Decoder, answer_cleansing

ModuleNotFoundError: No module named 'openai'

In [2]:
dataset = load_dataset("gsm8k", ignore_verifications=True)

/home/raj/.local/lib/python3.7/site-packages/datasets/load.py:1763: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  FutureWarning,
Found cached dataset parquet (/home/raj/.cache/huggingface/datasets/parquet/main-a198226013e23936/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|██████████| 2/2 [00:00<00:00, 804.82it/s]


In [3]:
data_train = dataset['train']
data_test = dataset['test']

In [4]:
def data_reader(args=None, data=None):

    questions = []
    answers = []
    
    for datum in data:
        questions.append(datum["question"].strip())
        answers.append(datum["answer"].split("#### ")[-1]) 
    
    q_len_list = []
    for q in questions:
        q_len_list.append(len(q.split(" ")))
    q_len_mean = mean(q_len_list)
    
    print("dataset : {}".format('gsm8k'))
    print("data size : {}".format(len(answers)))
    print("average num of words for each sample : {}".format(q_len_mean))
    
    return questions, answers

In [5]:
def main(args):
    
    print("OPENAI_API_KEY:")
    print(os.getenv("OPENAI_API_KEY"))
    
    # Initialize decoder class (load model and tokenizer) ...
    decoder = Decoder(args=args)
    
    print("setup data loader ...")
    data_ = data_test

    demo = create_demo_text(args, cot_flag=True)
    
    total = 0
    correct_list = []
            
    for i, data in enumerate(data_):
        print('*************************')
        print("{}st data".format(i+1))
                
        # Prepare question template ...
        x, y = data
        x = "Q: " + x[0] + "\n" + "A:"
        y = y[0].strip()
        
        if args.method == "zero_shot":
            x = x + " " + args.direct_answer_trigger_for_zeroshot
        elif args.method == "zero_shot_cot":
            x = x + " " + args.cot_trigger
        elif args.method == "few_shot":
            x = demo + x
        elif args.method == "few_shot_cot":
            x = demo + x
        else:
            raise ValueError("method is not properly defined ...")
        
        # Answer prediction by generating text ...
        max_length = 128 if "cot" in args.method else 32
        z = decoder.decode(args, x, max_length, i, 1)

        # Answer extraction for zero-shot-cot ...
        if args.method == "zero_shot_cot":
            z2 = x + z + " " + args.direct_answer_trigger_for_zeroshot_cot
            max_length = args.max_length_direct
            pred = decoder.decode(args, z2, max_length, i, 2)
            print(z2 + pred)
        else:
            pred = z
            print(x + pred)

        # Clensing of predicted answer ...
        pred = answer_cleansing(args, pred)
        
        # Choose the most frequent answer from the list ...
        print("pred : {}".format(pred))
        print("GT : " + y)
        print('*************************')
        
        # Checking answer ...
        correct = (np.array([pred]) == np.array([y])).sum().item()
        correct_list.append(correct)
        total += 1 #np.array([y]).size(0)
        
        if i%5 == 0:
            break
    
    # Calculate accuracy ...
    accuracy = (sum(correct_list) * 1.0 / total) * 100
    print("accuracy : {}".format(accuracy))
    

In [6]:
args = parse_arguments()
main(args)
    

usage: ipykernel_launcher.py [-h] [--api_log_file_name API_LOG_FILE_NAME]
                             [--random_seed RANDOM_SEED]
                             [--dataset {aqua,gsm8k,commonsensqa,addsub,multiarith,strategyqa,svamp,singleeq,bigbench_date,object_tracking,coin_flip,last_letters}]
                             [--minibatch_size {1}]
                             [--max_num_worker MAX_NUM_WORKER]
                             [--model {gpt3,gpt3-medium,gpt3-large,gpt3-xl}]
                             [--method {zero_shot,zero_shot_cot,few_shot,few_shot_cot}]
                             [--cot_trigger_no COT_TRIGGER_NO]
                             [--max_length_cot MAX_LENGTH_COT]
                             [--max_length_direct MAX_LENGTH_DIRECT]
                             [--limit_dataset_size LIMIT_DATASET_SIZE]
                             [--api_time_interval API_TIME_INTERVAL]
                             [--log_dir LOG_DIR]
ipykernel_launcher.py: error: unrecognize

SystemExit: 2

/home/raj/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("PROXY_API_KEY")
base_url = os.getenv("PROXY_BASE_URL")

print(api_key, base_url)

client = openai.OpenAI(api_key=api_key, base_url=base_url)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # model to use from Models Tab
    messages=[
        {"role": "user", "content": "this is a test request, write a short poem"}
    ],
)
print(response)

sk-UrWEfDNtKOBV9AZ-hLjA-Q https://cmu.litellm.ai
ChatCompletion(id='chatcmpl-9Fc59smDt6mwzsB8Ta31Vzt5wdhuc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="In a world of chaos and strife,\nA moment of calm in the stormy life.\nSoft whispers of the wind,\nBringing peace to the troubled mind.\nNature's beauty all around,\nA gentle solace that can be found.\nIn this fleeting moment of grace,\nWe find our hearts in a tranquil place.", role='assistant', function_call=None, tool_calls=None))], created=1713508335, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_d9767fc5b9', usage=CompletionUsage(completion_tokens=63, prompt_tokens=17, total_tokens=80))


In [ ]:
# lOAD LLAMA2-7b model from huggingface

from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained
